In [1]:
import radarsimpy

print("`RadarSimPy` used in this example is version: " + str(radarsimpy.__version__))

`RadarSimPy` used in this example is version: 12.6.1


# Impact of Vertical Multi-Path with Varying Radar Mounting Heights

## Introduction

In radar systems, accurate signal reception is crucial for reliable operation, particularly in applications such as automotive safety, aviation, and maritime navigation. One significant factor influencing radar performance is the vertical multi-path effect, where radar signals reflect off surfaces like the ground, causing interference patterns that can distort the received signal. The height at which radar sensors are mounted plays a critical role in determining the extent and nature of this interference. Understanding the impact of varying sensor mounting heights on the vertical multi-path effect is essential for optimizing radar system design and improving signal accuracy. This study explores how different mounting heights influence the multi-path effect, aiming to provide insights for enhancing radar performance across various applications.

In this example, we will employ [`RadarSimPy`](https://radarsimx.com/radarsimx/radarsimpy/)'s ray tracing capabilities to demonstrate how vertical multipath effects from the ground can impact the received signal amplitude in an FMCW radar system.

## Create Radar Model

Firstly, import the required modules from `radarsimpy`. `numpy` will also be needed in this example.


In [2]:
import numpy as np
from radarsimpy import Radar, Transmitter, Receiver

### Transmitter

Setup the basic transmitter parameters through [Transmitter](https://radarsimx.github.io/radarsimpy/radar.html#radarsimpy-transmitter) module.

#### Define a Radar Transmitter

As shown in the diagram below, `f` and `t` are used to define the waveform modulation. For a linear frequency-modulated continuous waveform (FMCW), it can be specified as `f = [fstart, fend]` and `t = [tstart, tend]`. If `t` is a single number `t = t0`, which is equivalent to `t = [0, t0]`. The bandwidth of the FMCW is `abs(fstart - fend)`. `prp` is the pulse repetition period, and `prp >= (tend - tstart)`.

```
|                       prp
|                  +-----------+
|
|            +---f[1]--->  /            /            /
|                         /            /            /
|                        /            /            /
|                       /            /            /
|                      /            /            /     ...
|                     /            /            /
|                    /            /            /
|                   /            /            /
|      +---f[0]--->/            /            /
|
|                  +-------+
|                 t[0]    t[1]
```


In [3]:
tx_channel = dict(
    location=(0, 0, 0),
)

tx = Transmitter(
    f=[76.5e9 - 80e6, 76.5e9 + 80e6],
    t=20e-6,
    tx_power=15,
    prp=100e-6,
    pulses=1,
    channels=[tx_channel],
)

### Receiver

Setup the receiver parameters through [Receiver](https://radarsimx.github.io/radarsimpy/radar.html#radarsimpy-receiver) module.

#### Define a Radar Receiver

- `fs` – Sampling rate (sps)
- `noise_figure` – Noise figure (dB)
- `rf_gain` – Total RF gain (dB)
- `load_resistor` – Load resistor to convert power to voltage (Ohm)
- `baseband_gain` – Total baseband gain (dB)


In [4]:
rx_channel = dict(
    location=(0, 0, 0),
)

rx = Receiver(
    fs=20e6,
    noise_figure=8,
    rf_gain=20,
    load_resistor=1000,
    baseband_gain=80,
    channels=[rx_channel],
)

### Radar System

Use the defined transmitter and receiver to create the radar system.


In [5]:
from radarsimpy.simulator import sim_radar
from scipy import signal
import radarsimpy.processing as proc
import time

radar = Radar(
    transmitter=tx, receiver=rx, location=(0, 0, 0)
)

target_1 = {"model": "../models/cr.stl", "location": (300, 0, 0), "speed": (-1, 0, 0)}

targets = [target_1]

data = sim_radar(radar, targets, density=0.5, frame_time=np.arange(0, 290, 1))
baseband = data["baseband"]

range_window = signal.windows.chebwin(radar.sample_prop["samples_per_pulse"], at=60)
range_profile = proc.range_fft(baseband, range_window)

amp_single = np.max(20 * np.log10(np.abs(range_profile)), axis=2)

In [6]:
mount_height = [0.2, 0.6, 1.0]

amp_multi = np.zeros((len(mount_height), 290), dtype=np.float64)

for idx, height in enumerate(mount_height):
    radar = Radar(
        transmitter=tx, receiver=rx, location=(0, 0, height), time=np.arange(0, 290, 1)
    )

    target_1 = {
        "model": "../models/cr.stl",
        "location": (300, 0, height),
        "speed": (-1, 0, 0),
        "rotation": (0, 0, 0),
    }

    target_2 = {
        "model": "../models/surface_400x400.stl",
        "location": (0, 0, 0),
        "speed": (0, 0, 0),
        "permittivity": 3.2 + 0.1j,
        "is_ground": True,
    }
    targets = [target_1, target_2]

    data = sim_radar(radar, targets, density=0.5, frame_time=np.arange(0, 290, 1))
    baseband = data["baseband"]

    range_window = signal.windows.chebwin(radar.sample_prop["samples_per_pulse"], at=60)
    range_profile = proc.range_fft(baseband, range_window)

    amp_multi[idx, :] = np.max(20 * np.log10(np.abs(range_profile)), axis=2).flatten()

## Results

Duo to the multi-path:

- Path 1: Tx -> Corner reflecor -> Rx
- Path 2: Tx -> Ground -> Corner reflecor -> Ground -> Rx
- Path 3: Tx -> Ground -> Corner reflecor -> Rx
- Path 4: Tx -> Corner reflecor -> Ground -> Rx

A fluctuation can be observed from the received target amplitude versus the target range.


In [7]:
import plotly.graph_objs as go
from IPython.display import Image

t_range = 10 + np.arange(290, 0, -1) * 1

fig = go.Figure()

for idx, height in enumerate(mount_height):
    fig.add_trace(
        go.Scatter(
            x=t_range,
            y=amp_multi[idx, :] - amp_single.flatten(),
            name=str(np.round(height, 1)) + " m mounting height",
        )
    )

fig.update_layout(
    yaxis=dict(title="Normalized RCS (dBsm)"),
    xaxis=dict(title="Range (m)"),
)

# uncomment this to display interactive plot
fig.show()

# display static image to reduce size on radarsimx.com
# img_bytes = fig.to_image(format="jpg", scale=2)
# display(Image(img_bytes))

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed